# PDF RAG - 소비자 분쟁조정 사례집 QA

## PDF Data

In [ ]:
import os
from glob import glob
pdf_files = glob(os.path.join('pdf_data', '*분쟁*.pdf'))
len(pdf_files) 

In [ ]:
# pdf 파일 목록
pdf_files

In [ ]:
# pdf 파일을 읽어서 텍스트로 변환
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdf_files[1])
data = loader.load()
len(data)

In [ ]:
# 첫번째 페이지의 텍스트 출력
from pprint import pprint
pprint(data[0].page_content) 

In [ ]:
# 첫번째 페이지의 메타데이터 출력
pprint(data[0].metadata)  

In [ ]:
# 10번째 페이지의 텍스트 출력
pprint(data[10].page_content)

In [ ]:
# 10번째 페이지의 텍스트에서 정보 추출
import re
split_pattern = r'사\n례\d+.*사건번호.*결정일자.*\d{4}\.\s?\d{1,2}\.\s?\d{1,2}\.'
split_text = re.findall(split_pattern, "".join(data[10].page_content))
split_text  

In [ ]:
# split_text가 존재하면 split_pattern으로 분리
if split_text:
    parts = re.split(split_pattern, "".join(data[10].page_content))

In [ ]:
# 분리된 텍스트 출력
parts[0]

In [ ]:
# 분리된 텍스트 출력
parts[1]

In [ ]:
# 특정 문자열의 위치 찾기
re.search(r'주 문\n', parts[1]).span()[0]

In [ ]:
# 제목 추출
title = parts[1][:re.search(r'주 문\n', parts[1]).span()[0]].strip()
title

In [ ]:
# 내용 추출
content = parts[1][re.search(r'주 문\n', parts[1]).span()[0]:]
content

In [ ]:
# 구분 문자열에서 사례 번호 추출
split_text[0]

In [ ]:
re.findall('례\s?(\d+)\s?사건번호', split_text[0])[0]

In [ ]:
# 사건에 대한 메타데이터 추출하는 Pydantic 스키마 정의

from langchain_core.pydantic_v1 import BaseModel, Field

class Case(BaseModel):
    case_number: str = Field(description="The number of the case", required=True)
    case_date: str = Field(description="The date when the case was occurred (year, month, day)", required=True)   

In [ ]:
# Extraction chain 구성
from langchain_core.prompts import PromptTemplate
from langchain_experimental.llms.ollama_functions import OllamaFunctions


prompt = PromptTemplate.from_template(
    """ Extract relevant information from the following text:
    
TEXT: {text} \n

AI: """
)

text = split_text[0]

llm = OllamaFunctions(model="llama3", format="json", temperature=0)
runnable = prompt | llm.with_structured_output(schema=Case)
response = runnable.invoke({"text": text})
print(response)

In [ ]:
dict(response)

In [ ]:
data[-2].page_content   

In [ ]:
# 문서 객체를 페이지별로 순회하며 사례 번호 등 메타데이터를 추출하고 업데이트
pdf_docs = []
case_metadata = {}
for doc in data[10:-2]:
    split_text = re.findall(split_pattern, "".join(doc.page_content))
    if split_text:

        # case id
        case_metadata['case_id'] = re.findall('례\s?(\d+)\s?사건번호', split_text[0])[0]

        parts = re.split(split_pattern, "".join(doc.page_content))

        if re.search(r'주 문\n', parts[1]):
            # Add title to metadata
            case_metadata['title'] = parts[1][:re.search(r'주 문\n', parts[1]).span()[0]].replace('\n', '').strip()
                    
            # Update content
            doc.page_content = parts[1][re.search(r'주 문\n', parts[1]).span()[0]:].strip()
        else:
            case_metadata['title'] = ''
            
        # Extract metadata from text
        i = 0
        while i < 10:
            try:
                response = runnable.invoke({"text": split_text[0]})
                for k, v in dict(response).items():
                    case_metadata[k] = v.replace("\n", "").replace(" ", "")
                break
            except:
                i += 1
                continue

        # Update metadata 
        doc.metadata.update(case_metadata)

        # Append to split_docs
        pdf_docs.append(doc)

    else:
        # Update metadata 
        doc.metadata.update(case_metadata) 

        # Append to split_docs
        pdf_docs.append(doc)

len(pdf_docs)

In [ ]:
pprint(pdf_docs[0].page_content)

In [ ]:
pprint(pdf_docs[0].metadata)

## Embedding

In [ ]:
# HugoingFace Embeddings를 다운로드
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
)

In [ ]:
# 문서를 임베딩
embeddings = embeddings_model.embed_documents(
    [
        "온라인 쇼핑몰에서 주문한 제품이 불량품으로 배송되었습니다. 이에 대한 법적 책임은 누구에게 있나요?",
        "구입한 전자제품이 고장나서 환불을 요청했지만 거부당했습니다. 피해 보상을 받을 수 있나요?",
        "호텔 예약 후 도착했는데 예약이 취소되었다고 했습니다. 이에 대한 대응 방법은 무엇인가요?",
        "자동차 수리 후 동일한 문제가 재발했습니다. 수리업체에 대한 법적 조치를 취할 수 있나요?",
        "항공편이 지연되어 중요한 일정을 놓쳤습니다. 이에 대한 피해 보상을 받을 수 있나요?"
    ]
)
len(embeddings), len(embeddings[0])

In [ ]:
embedded_query = embeddings_model.embed_query("에어컨 제품 불량에 대해서 보상을 받을 수 있을까요?")
embedded_query[:5]

In [ ]:
# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

# 쿼리와 문서 간의 코사인 유사도 계산
for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

## Chunking

In [ ]:
# HugoingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

text = "에어컨 제품 불량에 대해서 보상을 받을 수 있을까요?"
encoded = tokenizer.encode(text)
print(len(text), len(encoded))
print(encoded)

In [ ]:
# Token 수를 기준으ㄹ 문서를 청크 단위로 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 120,
    chunk_overlap  = 10,
)

split_docs = text_splitter.split_documents(pdf_docs)
print(len(split_docs))
print(split_docs[0])

In [ ]:
sample_text = split_docs[0].page_content
sample_encoded = tokenizer.encode(sample_text)
len(sample_text), len(sample_encoded)

In [ ]:
print(len(tokenizer.encode(split_docs[0].page_content)))
pprint(split_docs[0].page_content)

In [ ]:
print(len(tokenizer.encode(split_docs[1].page_content)))
pprint(split_docs[1].page_content)

In [ ]:
# 마침표 뒤에 나오는 줄바꿈 문자는 그대로 두고 나머지 줄바꿈 문자만 제거
result = re.sub(r'(?<!\.)\n', ' ', split_docs[1].page_content)
result

In [ ]:
pprint(split_docs[0].metadata)

In [ ]:
pprint(split_docs[1].metadata)

In [ ]:
f"### 이 사건은 '{split_docs[1].metadata['title']}'에 대한 사례입니다."

# Indexing

In [ ]:
final_docs = []
for doc in split_docs:
    doc.page_content = f"### 이 사건은 '{doc.metadata['title']}'에 대한 사례입니다.\n\n" + \
    re.sub(r'(?<!\.)\n', ' ', doc.page_content)
    final_docs.append(doc)

print(final_docs[0].page_content)

In [ ]:
print(final_docs[1].page_content)

In [ ]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=final_docs, 
                                    embedding=embeddings_model, 
                                    collection_name="consumer_case_qa",
                                    persist_directory="./chroma_db")

In [ ]:
chroma_docs = vectorstore.similarity_search("세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?", k=5)
for doc in chroma_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:200])

## Retrieval

In [ ]:
# Top K
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)

query = "세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:100])

In [ ]:
# metadata를 이용한 필터링

retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': 5,
        'filter': {'case_id':'01'}
        }
)


query = "세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:100])

In [ ]:
# page_content를 이용한 필터링

retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': 5,
        'where_document': {'$contains': '세탁'}
        }
)

query = "세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:100])

## Generation

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

In [ ]:
# RAG Chain
llm = ChatOllama(model="qwen2", temperature=0)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

item = "세탁"
query = f"{item} 불량에 대한 손해배상은 어떻게 이루어지나요?"

retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': 2,
        'where_document': {'$contains': item}
        }
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


response = rag_chain.invoke(query)
response

In [ ]:
print(response)